In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing common libraries  
import os , random
import datetime
import numpy as np # linear algebra
import pandas as pd # data processing, 
import matplotlib.pyplot as plt # ploting
import seaborn as sns # visualisation 
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
import catboost as cat
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')
sns.set_style("ticks")
import optuna
from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold, KFold


In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv", low_memory=False)
train["date_time"] = pd.to_datetime(train["date_time"])
test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv", low_memory=False)
test["date_time"] = pd.to_datetime(test["date_time"])

In [ ]:
import seaborn as sns

sns.displot(x="deg_C", data=train,bins=3)



In [ ]:
def newfeatures(df):
    df["hour"] = df["date_time"].dt.hour
    df["working_hours"] =  df["hour"].isin(np.arange(8, 21, 1)).astype("int")
    df["is_weekend"] = (df["date_time"].dt.dayofweek >= 5).astype("int")

    df["SMC"] = (df["absolute_humidity"] * 100) / df["relative_humidity"]
    df['Dew_Point'] = 243.12*(np.log(df['relative_humidity'] * 0.01) + (17.62 *     df['deg_C'])/(243.12+df['deg_C']))/(17.62-(np.log(df['relative_humidity'] * 0.01)+17.62*df['deg_C']/(243.12+df['deg_C'])))
    df['SMC'] = (df['absolute_humidity'] * 100) / df['relative_humidity']
    df['temperature_lag_3'] = df['deg_C'] - df['deg_C'].shift(periods=3, fill_value=0)
    df['temperature_lag_6'] = df['deg_C'] - df['deg_C'].shift(periods=6, fill_value=0)
    df['Partial_pressure'] = 243.12*(np.log(df['absolute_humidity'] * 0.01) + (17.62 * df['deg_C'])/(243.12+df['deg_C']))/(17.62-(np.log(df['relative_humidity'] * 0.01)+17.62*df['deg_C']/(243.12+df['deg_C'])))
    df['date_time'] = df['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9
    df['S5-S1']=df['sensor_5']-df['sensor_1']
    df['S1-S4']=df['sensor_1']-df['sensor_4']
    df['S4-S2']=df['sensor_4']-df['sensor_2']
    df['S5-S2']=df['sensor_5']-df['sensor_2']
    df['S1-S2']=df['sensor_1']-df['sensor_3']
    conditions = [
    (df['deg_C'] <= 12),
    (df['deg_C'] > 12) & (df['deg_C'] <= 22),
    (df['deg_C'] > 22)
    ]

#create a list of the values we want to assign for each condition
    values = [0,1,2]

#create a new column and use np.select to assign values to it using our lists as arguments
    df['temp_tier'] = np.select(conditions, values)
    return df


In [ ]:
train=newfeatures(train)
test=newfeatures(test)
test.head()

In [ ]:
import seaborn as sns
ax = sns.boxplot(y="temp_tier", data=train)


In [ ]:
sns.displot(train, x="target_nitrogen_oxides", kind="kde", cut=0)   

ax = sns.boxplot(y="target_benzene", data=train)

In [ ]:
train = train.drop([7110], axis = 0)

for i in ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']:
    train[i] = np.log1p(train[i])


y1 = train['target_carbon_monoxide']
y2 = train['target_benzene']
y3 = train['target_nitrogen_oxides']

X = train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'], axis = 1)
X1 = train.drop(['target_benzene', 'target_nitrogen_oxides'], axis = 1)
X2 = train.drop(['target_nitrogen_oxides'], axis = 1)



In [ ]:

y3.head()

### X2.columns

# Optuna parameters for each target (I change here only targets)

def objective(trial, data = X, target = y3):

    params = {
        'depth': trial.suggest_int('depth', 2, 6),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'iterations': trial.suggest_int('iterations', 500, 10000),
        'max_bin': trial.suggest_int('max_bin', 1, 300),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 300),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log = True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10.0),
        'subsample': trial.suggest_float('subsample', 0.1, 0.8),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
        'cat_features': ['hour', 'working_hours', 'is_weekend','temp_tier'],
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE'
    }
    
    model = CatBoostRegressor(**params)
    scores = []
    k = KFold(n_splits = 4, random_state = 228, shuffle = True)
    for i, (trn_idx, val_idx) in enumerate(k.split(X)):
        
        X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
        y_train, y_val = y3.iloc[trn_idx], y3.iloc[val_idx]

        model.fit(X, y3, eval_set = [(X_val, y_val)], early_stopping_rounds = 30, verbose = False, use_best_model = True)
        
        tr_preds = np.expm1(model.predict(X_train))
        tr_score = np.sqrt(mean_squared_log_error(np.expm1(y_train), tr_preds))
        
        val_preds = np.expm1(model.predict(X_val))
        val_score = np.sqrt(mean_squared_log_error(np.expm1(y_val), val_preds))

        scores.append((tr_score, val_score))
        
        print(f"Fold {i} | RMSLE: {val_score}")
        
        
    scores = pd.DataFrame(scores, columns = ['train score', 'validation score'])
    
    return scores['validation score'].mean()

study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 20)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)


y3

 {'depth': 5, 'learning_rate': 0.1336172575126029, 'iterations': 9870, 'max_bin': 85, 'min_data_in_leaf': 176, 'l2_leaf_reg': 0.47255556409463717, 'bagging_temperature': 6.290468771397568, 'subsample': 0.7337538866725786, 'grow_policy': 'SymmetricTree', 'leaf_estimation_method': 'Gradient'}
Best value: 0.004481325552645856

y2

Number of finished trials: 20
Best trial: {'depth': 5, 'learning_rate': 0.16922898702324848, 'iterations': 9624, 'max_bin': 187, 'min_data_in_leaf': 119, 'l2_leaf_reg': 0.15692495225226663, 'bagging_temperature': 0.11489860045608591, 'subsample': 0.518476242344081, 'grow_policy': 'SymmetricTree', 'leaf_estimation_method': 'Newton'}
Best value: 0.0025068183601230816

Fold 3 | RMSLE: 0.004316732916015073
Number of finished trials: 20
Best trial: {'depth': 5, 'learning_rate': 0.1723956529272847, 'iterations': 8235, 'max_bin': 202, 'min_data_in_leaf': 13, 'l2_leaf_reg': 0.025988660512111194, 'bagging_temperature': 3.126276416033602, 'subsample': 0.7799317755054309, 'grow_policy': 'Depthwise', 'leaf_estimation_method': 'Newton'}
Best value: 0.004294154550650668

In [ ]:
# Carbon monoxide
# Mean RMSLE on 5 Folds with Optuna - 0.0105
paramsCB1 = {'depth': 5, 
              'learning_rate': 0.1723956529272847, 
              'iterations': 8235,
              'max_bin': 202,
              'min_data_in_leaf': 13, 
              'l2_leaf_reg': 0.025988660512111194, 
              'bagging_temperature': 3.126276416033602, 
              'subsample': 0.7799317755054309,
              'grow_policy': 'Depthwise',
              'leaf_estimation_method': 'Newton',
             'loss_function': 'RMSE',
             'eval_metric': 'RMSE',
             'cat_features': ['hour', 'working_hours', 'is_weekend','temp_tier']}

# Benzene
# Mean RMSLE on 5 Folds with Optuna - 0.003
paramsCB2 = {'depth': 5,
             'learning_rate': 0.16922898702324848,
             'iterations': 9624,
             'max_bin': 187,
             'min_data_in_leaf': 119,
             'l2_leaf_reg': 0.15692495225226663, 
             'bagging_temperature': 0.11489860045608591,
             'subsample': 0.518476242344081,
             'grow_policy': 'SymmetricTree',
             'leaf_estimation_method': 'Newton',
             'loss_function': 'RMSE',
             'eval_metric': 'RMSE',
             'cat_features': ['hour', 'working_hours', 'is_weekend','temp_tier']}

# Nitrogen oxides
# Mean RMSLE on 5 Folds with Optuna - 0.021
paramsCB3 = {'depth': 5, 
              'learning_rate': 0.1336172575126029,
              'iterations': 9870, 
              'max_bin': 85,
              'min_data_in_leaf': 176, 
              'l2_leaf_reg': 0.47255556409463717,
              'bagging_temperature': 6.290468771397568,
              'subsample': 0.7337538866725786,
              'grow_policy': 'SymmetricTree', 
              'leaf_estimation_method': 'Gradient',
             'loss_function': 'RMSE',
             'eval_metric': 'RMSE',
             'cat_features': ['hour', 'working_hours', 'is_weekend','temp_tier']}


In [ ]:
ss=pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

In [ ]:
s1=pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')

In [ ]:
ss.head()

In [ ]:
folds = KFold(n_splits = 4, random_state = 228, shuffle = True)

predictions = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(folds.split(X, y1)):
    
    X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y1.iloc[trn_idx], y1.iloc[val_idx]

    model = CatBoostRegressor(**paramsCB1)
   
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False, early_stopping_rounds = 30, use_best_model = True)
    
    predictions += model.predict(test) / folds.n_splits 
    
ss['target_carbon_monoxide'] = np.expm1(predictions)




In [ ]:
folds = KFold(n_splits = 4, random_state = 228, shuffle = True)

predictions = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(folds.split(X, y2)):
    
    X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y2.iloc[trn_idx], y2.iloc[val_idx]

    model = CatBoostRegressor(**paramsCB2)
   
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False, early_stopping_rounds = 30, use_best_model = True)
    
    predictions += model.predict(test) / folds.n_splits 
    
ss['target_benzene'] = np.expm1(predictions)



In [ ]:
folds = KFold(n_splits = 4, random_state = 228, shuffle = True)

predictions = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(folds.split(X, y3)):
    
    X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y3.iloc[trn_idx], y3.iloc[val_idx]

    model = CatBoostRegressor(**paramsCB3)
   
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False, early_stopping_rounds = 30, use_best_model = True)
    
    predictions += model.predict(test) / folds.n_splits 
    
ss['target_nitrogen_oxides'] = np.expm1(predictions)

In [ ]:
ss.to_csv('submission.csv',index=False)

In [ ]:
ss.head()